In [52]:
import pandas as pd
import pyodbc
import numpy as np
from sqlalchemy import create_engine


### Import Data

In [53]:
file_path = r'C:\Users\SHawe\OneDrive\Documents\Open Uni\Level 3\TM470\Data\Power Queries\SPAN Combined Data Power Query.xlsx'
pop_file_path = r'C:\Users\SHawe\OneDrive\Documents\Open Uni\Level 3\TM470\Data\2021-06-30 HB Populations Mid 2021.xlsx'

In [54]:
workload_df = pd.read_excel(file_path, sheet_name='Workload_Production_Combined')
turnaround_df = pd.read_excel(file_path, sheet_name='Turnaround_Combined')
medical_df = pd.read_excel(file_path, sheet_name='Medical_Combined')
non_medical_df = pd.read_excel(file_path, sheet_name='Non Medical_Combined')
consumables_df = pd.read_excel(file_path, sheet_name='Consumables_Combined')
populations_df = pd.read_excel(pop_file_path, sheet_name='Population')

In [55]:
populations_df = populations_df[['Health Board (Code)', 'Health Board', 'HB Population']]

### Final Data Cleaning

In [56]:
column_renaming = {
    'Health Board (Code)': 'health_board_code', 
    'Health Board': 'health_board_name', 
    'HB Population': 'population'
}
populations_df = populations_df.rename(columns=column_renaming)

In [57]:
# replace na values with 0
workload_df['Value'] = workload_df['Value'].fillna(0)

In [58]:
# replace values to ensure consistency
medical_df['Area'] = medical_df['Area'].replace({'Mortuary & Autopsy (Adult)': 'Mortuary & Autopsy',
                                                 'Mortuary & Autopsy (Perinatal)': 'Mortuary & Autopsy',
                                                 'Non-Cervical/ Diagnostic Cytopathology': 'Non-Cervical/Diagnostic Cytopathology',
                                                 'Cervical Screening Cytopathology': 'Cervical Screening'})

In [59]:
# add the role types to each dataframe
medical_df['role_type'] = 'Medical'
non_medical_df['role_type'] = 'Non-Medical'

In [60]:
# renaming column appropriately
medical_df = medical_df.rename(columns={'Category': 'Role',
                                        'Value': 'Headcount'})

In [61]:
# filter for consultant data only
medical_df = medical_df[(medical_df['Role'] == 'Consultant') |
                        (medical_df['Role'] == 'Non-Consultant (SAS)') |
                        (medical_df['Role'] == 'University Consultant')]

In [62]:
# ensure strings are consistent with cases
medical_df.loc[:, 'Sub-Category'] = medical_df['Sub-Category'].str.title()
turnaround_df.loc[:, 'Type'] = turnaround_df['Type'].str.title()

In [63]:
# assign vacancies to a separate dataframe and add a vacancy/post column
medical_vac_df = medical_df[(medical_df['Sub-Category'] == 'Consultant Vacancies (Approved Vacancies - Headcount)') |
                        (medical_df['Sub-Category'] == 'Non-Consultant (Sas) Vacancies (Approved Vacancies - Headcount)') |
                        (medical_df['Sub-Category'] == 'University Consultant Vacancies (Approved Vacancies - Headcount)')]
medical_vac_df['Post/Vacancy'] = 'Vacancy'


medical_post_df = medical_df[(medical_df['Sub-Category'] == 'Consultants In Post (Headcount)') |
                        (medical_df['Sub-Category'] == 'Non-Consultant (Sas) In Post (Headcount)') |
                        (medical_df['Sub-Category'] == 'University Consultants In Post (Headcount)')]
medical_post_df['Post/Vacancy'] = 'Post'

C:\Users\SHawe\AppData\Local\Temp\ipykernel_12196\3933163389.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_vac_df['Post/Vacancy'] = 'Vacancy'
C:\Users\SHawe\AppData\Local\Temp\ipykernel_12196\3933163389.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_post_df['Post/Vacancy'] = 'Post'


In [64]:
# merge the data again
medical_df = pd.concat([medical_post_df, medical_vac_df])

In [65]:
# column is no longer required
medical_df = medical_df.drop(columns=['Sub-Category'])

In [66]:
# merge all staffing
staffing_df = pd.concat([medical_df, non_medical_df])

In [67]:
# convert column types
staffing_df['Headcount'] = staffing_df['Headcount'].fillna(0).astype(int)
staffing_df['WTE'] = pd.to_numeric(staffing_df['WTE'], errors='coerce')
staffing_df['WTE'] = staffing_df['WTE'].round(2)
staffing_df['WTE'].fillna(0.0, inplace=True)
turnaround_df['Value'].fillna(0.0, inplace=True)

In [68]:
# strip white space
staffing_df.loc[:, 'Role'] = staffing_df['Role'].str.strip()
staffing_df.loc[:, 'Band'] = staffing_df['Band'].str.strip()

In [69]:
# medical staff don't have a band, remove na values and add None
staffing_df['Band'] = staffing_df['Band'].fillna('None')

In [70]:
# remove invalid bands
staffing_df = staffing_df[~((staffing_df['Band'] == 'Other Type') |
                          (staffing_df['Band'] == 'Other Staff') |
                          (staffing_df['Band'] == 'Other Band'))]

In [71]:
# drop rows where value is 0
consumables_df = consumables_df[consumables_df['Value'] != 0]
workload_df = workload_df[workload_df['Value'] != 0]
turnaround_df = turnaround_df[turnaround_df['Value'] != 0]

### Connect to database

In [72]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-OG2GI91;'
                      'Database=pathology;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

### Define unique key values

In [73]:
unique_years = workload_df['Year'].unique()
unique_activity_areas = workload_df['Area'].unique()
unique_activity_categories = workload_df['Category'].unique()
unique_turnaround_categories = turnaround_df['Category'].unique()
unique_turnaround_areas = turnaround_df['Area'].unique()
unique_turnaround_types = turnaround_df['Type'].unique()
unique_test_types = workload_df['Type'].unique()
unique_roles = staffing_df['Role'].unique()
unique_role_types = ['Medical', 'Non-Medical']
unique_bands = staffing_df['Band'].unique()
unique_disciplines = non_medical_df['Area'].unique()
unique_consumable_areas = consumables_df['Area'].unique()

### Define functions for inserting into database

In [74]:
# function for inserting health board population data
def insert_health_boards(dataframe, cursor):
    for index, row in dataframe.iterrows():
        cursor.execute("""
            INSERT INTO dim_health_boards (health_board_code, health_board_name, population)
            VALUES (?, ?, ?)
            """, (row['health_board_code'], row['health_board_name'], row['population']))
    conn.commit()
    
    
# function for inserting health board population data
def insert_years(data, cursor):
    for index in unique_years:
        cursor.execute("""
            INSERT INTO dim_years (financial_year)
            VALUES (?)
            """, (index))
    conn.commit()
    

# function for inserting health board population data
def insert_activity_areas(data, cursor):
    for index in unique_activity_areas:
        cursor.execute("""
            INSERT INTO dim_activity_areas (activity_area_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_activity_categories(data, cursor):
    for index in unique_activity_categories:
        cursor.execute("""
            INSERT INTO dim_activity_categories (activity_category_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_test_types(data, cursor):
    for index in unique_test_types:
        cursor.execute("""
            INSERT INTO dim_test_types (test_type_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_turnaround_categories(data, cursor):
    for index in unique_turnaround_categories:
        cursor.execute("""
            INSERT INTO dim_turnaround_categories (turnaround_category_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_turnaround_areas(data, cursor):
    for index in unique_turnaround_areas:
        cursor.execute("""
            INSERT INTO dim_turnaround_areas (turnaround_area_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_turnaround_types(data, cursor):
    for index in unique_turnaround_types:
        cursor.execute("""
            INSERT INTO dim_turnaround_types (turnaround_type_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_roles(data, cursor):
    for index in unique_roles:
        cursor.execute("""
            INSERT INTO dim_roles (role_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_role_types(data, cursor):
    for index in unique_role_types:
        cursor.execute("""
            INSERT INTO dim_role_types (role_type_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_bands(data, cursor):
    for index in unique_bands:
        cursor.execute("""
            INSERT INTO dim_bands (band_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_disciplines(data, cursor):
    for index in unique_disciplines:
        cursor.execute("""
            INSERT INTO dim_disciplines (discipline_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_consumable_areas(data, cursor):
    for index in unique_consumable_areas:
        cursor.execute("""
            INSERT INTO dim_consumable_areas (area_name)
            VALUES (?)
            """, (index))
    conn.commit()
    
    
# function for inserting health board population data
def insert_health_boards(dataframe, cursor):
    for index, row in dataframe.iterrows():
        cursor.execute("""
            INSERT INTO dim_health_boards (health_board_code, health_board_name, population)
            VALUES (?, ?, ?)
            """, (row['health_board_code'], row['health_board_name'], row['population']))
    conn.commit()
    
    
# function for inserting health board population data
def insert_workload_production(dataframe, cursor):
    for index, row in dataframe.iterrows():
        cursor.execute("""
            INSERT INTO fact_workload_production (health_board_id, year_id, activity_area_id, activity_category_id,
                                                  test_type_id, amount)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (row['health_board_id'], row['year_id'], row['activity_area_id'],
                  row['activity_category_id'], row['test_type_id'], row['Value']))
    conn.commit()
    
    
# function for inserting health board population data
def insert_staffing(dataframe, cursor):
    for index, row in dataframe.iterrows():
        cursor.execute("""
            INSERT INTO fact_staffing (health_board_id, year_id, role_type_id, role_id, discipline_id,
                                                  band_id, post_or_vacancy, headcount, wte)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (row['health_board_id'], row['year_id'], row['role_type_id'], row['role_id'], row['discipline_id'],
                  row['band_id'], row['Post/Vacancy'], row['Headcount'], row['WTE']))
    conn.commit()
    

# function for inserting health board population data
def insert_turnaround(dataframe, cursor):
    for index, row in dataframe.iterrows():
        cursor.execute("""
            INSERT INTO fact_turnaround (health_board_id, year_id, turnaround_area_id, turnaround_category_id,
                                       turnaround_type_id, value)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (row['health_board_id'], row['year_id'], row['turnaround_area_id'], row['turnaround_category_id'],
                  row['turnaround_type_id'], row['Value']))
    conn.commit()
    

# function for inserting health board population data
def insert_consumables(dataframe, cursor):
    for index, row in dataframe.iterrows():
        cursor.execute("""
            INSERT INTO fact_consumable_costs (health_board_id, year_id, amount)
            VALUES (?, ?, ?)
            """, (row['health_board_id'], row['year_id'], row['Value']))
    conn.commit()

#### Call the functions to carry out database dimension tables ingesting

In [75]:
insert_health_boards(populations_df, cursor)
insert_years(unique_years, cursor)
insert_activity_areas(unique_activity_areas, cursor)
insert_activity_categories(unique_activity_categories, cursor)
insert_test_types(unique_test_types, cursor)
insert_turnaround_categories(unique_turnaround_categories, cursor)
insert_turnaround_areas(unique_turnaround_areas, cursor)
insert_turnaround_types(unique_turnaround_types, cursor)
insert_roles(unique_roles, cursor)
insert_role_types(unique_role_types, cursor)
insert_bands(unique_bands, cursor)
insert_disciplines(unique_disciplines, cursor)
insert_consumable_areas(unique_consumable_areas, cursor)

### Map values in fact data to corresponding foreign key values

In [76]:
# define a connection to the database
connection_string = (
    'mssql+pyodbc://@DESKTOP-OG2GI91/pathology?'
    'driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'
)

# create the SQLAlchemy engine
engine = create_engine(connection_string)

In [77]:
# retrieve health board ids
query = "SELECT health_board_id, health_board_code FROM dim_health_boards"
health_boards_df = pd.read_sql(query, engine)

# merge to add the id column
workload_df = workload_df.merge(health_boards_df, how='left', left_on='Health Board', right_on='health_board_code')
staffing_df = staffing_df.merge(health_boards_df, how='left', left_on='Health Board', right_on='health_board_code')
turnaround_df = turnaround_df.merge(health_boards_df, how='left', left_on='Health Board', right_on='health_board_code')
consumables_df = consumables_df.merge(health_boards_df, how='left', left_on='Health Board', right_on='health_board_code')

In [78]:
# retrieve year ids
query = "SELECT * FROM dim_years"
years_df = pd.read_sql(query, engine)

# merge to add the id column
workload_df = workload_df.merge(years_df, how='left', left_on='Year', right_on='financial_year')
staffing_df = staffing_df.merge(years_df, how='left', left_on='Year', right_on='financial_year')
turnaround_df = turnaround_df.merge(years_df, how='left', left_on='Year', right_on='financial_year')
consumables_df = consumables_df.merge(years_df, how='left', left_on='Year', right_on='financial_year')

In [79]:
# retrieve activity area ids
query = "SELECT * FROM dim_activity_areas"
activity_areas_df = pd.read_sql(query, engine)


# merge to add the id column
workload_df = workload_df.merge(activity_areas_df, how='left', left_on='Area', right_on='activity_area_name')

In [80]:
# retrieve activity category ids
query = "SELECT * FROM dim_activity_categories"
activity_categories_df = pd.read_sql(query, engine)


# merge to add the id column
workload_df = workload_df.merge(activity_categories_df, how='left', left_on='Category', right_on='activity_category_name')

In [81]:
# retrieve activity category ids
query = "SELECT * FROM dim_test_types"
test_types_df = pd.read_sql(query, engine)


# merge to add the id column
workload_df = workload_df.merge(test_types_df, how='left', left_on='Type', right_on='test_type_name')

In [82]:
# retrieve role ids
query = "SELECT * FROM dim_roles"
roles_df = pd.read_sql(query, engine)


# merge to add the id column
staffing_df = staffing_df.merge(roles_df, how='left', left_on='Role', right_on='role_name')

In [83]:
# retrieve role type ids
query = "SELECT * FROM dim_role_types"
role_types_df = pd.read_sql(query, engine)


# merge to add the id column
staffing_df = staffing_df.merge(role_types_df, how='left', left_on='role_type', right_on='role_type_name')

In [84]:
# retrieve band ids
query = "SELECT * FROM dim_bands"
bands_df = pd.read_sql(query, engine)


# merge to add the id column
staffing_df = staffing_df.merge(bands_df, how='left', left_on='Band', right_on='band_name')

In [85]:
# retrieve discipline ids
query = "SELECT * FROM dim_disciplines"
discipline_df = pd.read_sql(query, engine)


# merge to add the id column
staffing_df = staffing_df.merge(discipline_df, how='left', left_on='Area', right_on='discipline_name')

In [86]:
# retrieve turnaround area ids
query = "SELECT * FROM dim_turnaround_areas"
turnaround_areas_df = pd.read_sql(query, engine)


# merge to add the id column
turnaround_df = turnaround_df.merge(turnaround_areas_df, how='left', left_on='Area', right_on='turnaround_area_name')

In [87]:
# retrieve turnaround category ids
query = "SELECT * FROM dim_turnaround_categories"
turnaround_categories_df = pd.read_sql(query, engine)


# merge to add the id column
turnaround_df = turnaround_df.merge(turnaround_categories_df, how='left', left_on='Category', right_on='turnaround_category_name')

In [88]:
# retrieve turnaround category ids
query = "SELECT * FROM dim_turnaround_types"
turnaround_types_df = pd.read_sql(query, engine)


# merge to add the id column
turnaround_df = turnaround_df.merge(turnaround_types_df, how='left', left_on='Type', right_on='turnaround_type_name')

In [89]:
# retrieve turnaround category ids
query = "SELECT * FROM dim_consumable_areas"
consumable_areas_df = pd.read_sql(query, engine)


# merge to add the id column
consumables_df = consumables_df.merge(consumable_areas_df, how='left', left_on='Area', right_on='area_name')

#### Call the functions to carry out database fact tables ingesting

In [ ]:
insert_workload_production(workload_df, cursor)
insert_staffing(staffing_df, cursor)
insert_turnaround(turnaround_df, cursor)
insert_consumables(consumables_df, cursor)

### Close the database connetion

In [ ]:
cursor.close()
conn.close()